In [12]:
import pandas as pd
import numpy as np

In [13]:
seed = 100

In [14]:
def create_labels(train_splits, datasets):
    
    ehl = pd.read_csv('labels/base_ehl_edited.txt', sep="GGGG", header=None)
    ehl.columns = ["line"]
    ehl['type'] = 'train'
    
    ehl['source'] = ehl['line'].apply(lambda x: x.split(' ')[-1])
    ehl['condition'] = ehl['line'].apply(lambda x: x.split(' ')[-2])
    ehl['path'] = ehl['line'].apply(lambda x: x.split(' ')[-3])
    ehl['id'] = ehl['source'].str.cat(ehl['path'])
    
    fusion = pd.read_csv('labels/base_fusion.txt', sep="GGGG", header=None)
    fusion.columns = ["line"]
    fusion['type'] = 'test'
    
    fusion['source'] = fusion['line'].apply(lambda x: x.split(' ')[-1])
    fusion['condition'] = fusion['line'].apply(lambda x: x.split(' ')[-2])
    fusion['path'] = fusion['line'].apply(lambda x: x.split(' ')[-3])
    fusion['id'] = fusion['source'].str.cat(fusion['path'])
    
    for dataset in datasets:
        for train_split in train_splits:
            if dataset == 'fusion':
                proc = fusion[['id', 'path', 'condition']]
            else:
                proc = ehl[['id', 'path', 'condition']]
            train = proc.sample(frac=train_split, random_state=seed)
            test = pd.concat([train,proc]).drop_duplicates(keep=False)
            train.to_csv('labels/train_{}_{}.txt'.format(dataset, train_split), sep=' ', index=False, header=None)
            test.to_csv('labels/test_{}_{}.txt'.format(dataset, train_split), sep=' ', index=False, header=None)
    return 0

In [15]:
def create_tasks(train_splits, datasets):
    length = len(train_splits) * len(datasets)
    df = pd.DataFrame({
        'model': np.full(length, 'models/COVIDNet-CXR-Gisli'),
        'labels': np.array(['{}_{}'.format(j,i) for i in train_splits for j in datasets]),
        'classweights_normal': np.ones(length),
        'classweights_pneumonia': np.ones(length),
        'classweights_covid19': np.ones(length),
        'covid_percent': np.full(length, 0.5)
    })
    
    return 0

In [16]:
train_splits = np.array([0.9, 0.8, 0.7])
datasets = np.array(['ehl_edited', 'fusion'])

create_labels(train_splits, datasets)
create_tasks(train_splits, datasets)

/Users/gisli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/gisli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if sys.path[0] == '':


0

In [40]:
train_splits = np.array([0.9, 0.8, 0.7])
datasets = np.array(['ehl_edited', 'fusion'])
length = len(train_splits) * len(datasets)

df = pd.DataFrame({
    'model': np.full(length, 'models/COVIDNet-CXR-Gisli'),
    'labels': np.array(['{}_{}'.format(j,i) for i in train_splits for j in datasets]),
    'classweights_normal': np.ones(length),
    'classweights_pneumonia': np.ones(length),
    'classweights_covid19': np.ones(length),
    'covid_percent': np.full(length, 0.5)
})

In [41]:
df

,model,labels,classweights_normal,classweights_pneumonia,classweights_covid19,covid_percent
0,models/COVIDNet-CXR-Gisli,ehl_edited_0.9,1.0,1.0,1.0,0.5
1,models/COVIDNet-CXR-Gisli,fusion_0.9,1.0,1.0,1.0,0.5
2,models/COVIDNet-CXR-Gisli,ehl_edited_0.8,1.0,1.0,1.0,0.5
3,models/COVIDNet-CXR-Gisli,fusion_0.8,1.0,1.0,1.0,0.5
4,models/COVIDNet-CXR-Gisli,ehl_edited_0.7,1.0,1.0,1.0,0.5
5,models/COVIDNet-CXR-Gisli,fusion_0.7,1.0,1.0,1.0,0.5


In [43]:
df.to_csv('training_joblist.csv', index=False)

In [45]:
df = pd.read_csv('training_joblist.csv')

df

,model,labels,classweights_normal,classweights_pneumonia,classweights_covid19,covid_percent
0,models/COVIDNet-CXR-Gisli,ehl_edited_0.9,1.0,1.0,1.0,0.5
1,models/COVIDNet-CXR-Gisli,fusion_0.9,1.0,1.0,1.0,0.5
2,models/COVIDNet-CXR-Gisli,ehl_edited_0.8,1.0,1.0,1.0,0.5
3,models/COVIDNet-CXR-Gisli,fusion_0.8,1.0,1.0,1.0,0.5
4,models/COVIDNet-CXR-Gisli,ehl_edited_0.7,1.0,1.0,1.0,0.5
5,models/COVIDNet-CXR-Gisli,fusion_0.7,1.0,1.0,1.0,0.5


In [18]:
'train_{}.txt'.format(df.iloc[0]['labels'])

'train_ehl_edited.txt'

In [30]:
train_splits = np.array([0.9, 0.8, 0.7])
datasets = np.array(['ehl_edited', 'fusion'])

np.array(['{}_{}'.format(j,i) for i in train_splits for j in datasets])

array(['ehl_edited_0.9', 'fusion_0.9', 'ehl_edited_0.8', 'fusion_0.8',
       'ehl_edited_0.7', 'fusion_0.7'], dtype='<U14')

In [35]:
length

6

In [1]:
import pandas as pd

df = pd.read_csv('labels/test_ehl_edited_0.8.txt', sep="GGGG", header=None)
df.columns = ["line"]
#df['type'] = 'train'

#df['source'] = df['line'].apply(lambda x: x.split(' ')[-2])
df['condition'] = df['line'].apply(lambda x: x.split(' ')[-1])
df['path'] = df['line'].apply(lambda x: x.split(' ')[-3])
df['id'] = df['condition'].str.cat(df['path'])

/Users/gisli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/gisli/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [2]:
df.sample(5)

,line,condition,path,id
326,ehlpatient06732-study1-view1_frontal.jpg patie...,normal,ehlpatient06732-study1-view1_frontal.jpg,normalehlpatient06732-study1-view1_frontal.jpg
59,ehlNoFinding-patient16184-study1-view1_frontal...,normal,ehlNoFinding-patient16184-study1-view1_frontal...,normalehlNoFinding-patient16184-study1-view1_f...
58,ehlNoFinding-patient16629-study1-view1_frontal...,normal,ehlNoFinding-patient16629-study1-view1_frontal...,normalehlNoFinding-patient16629-study1-view1_f...
117,ehlpatient07254-study1-view1_frontal.jpg patie...,normal,ehlpatient07254-study1-view1_frontal.jpg,normalehlpatient07254-study1-view1_frontal.jpg
214,ehlpatient00608-study1-view1_frontal.jpg patie...,normal,ehlpatient00608-study1-view1_frontal.jpg,normalehlpatient00608-study1-view1_frontal.jpg


In [3]:
df['condition'].value_counts(normalize=True)

normal       0.854875
COVID-19     0.086168
pneumonia    0.058957
Name: condition, dtype: float64

In [4]:
df['condition'].value_counts()

normal       377
COVID-19      38
pneumonia     26
Name: condition, dtype: int64

In [1]:
1+1

2

In [19]:
import pandas as pd

df = pd.read_csv('labels/test_ehl_edited_0.8.txt', sep="GGGG", header=None)
df.columns = ["line"]
#df['type'] = 'train'

#df['source'] = df['line'].apply(lambda x: x.split(' ')[-2])

df['condition'] = df['line'].apply(lambda x: x.split(' ')[-1])
df['path'] = df['line'].apply(lambda x: x.split(' ')[-2])
df['id'] = df['condition'].str.cat(df['path'])

/Users/gisli/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
df.head()

,line,condition,path,id
0,ehlCOVID-19_02132021(8).jpg COVID-19_02132021(...,COVID-19,COVID-19_02132021(8).jpg,COVID-19COVID-19_02132021(8).jpg
1,ehlCOVID-19_02132021(47).jpg COVID-19_02132021...,COVID-19,COVID-19_02132021(47).jpg,COVID-19COVID-19_02132021(47).jpg
2,ehl4-x-day4.jpg 4-x-day4.jpg COVID-19,COVID-19,4-x-day4.jpg,COVID-194-x-day4.jpg
3,ehlCOVID-19_02132021(43).jpg COVID-19_02132021...,COVID-19,COVID-19_02132021(43).jpg,COVID-19COVID-19_02132021(43).jpg
4,ehlCOVID-1912132020(76).jpg COVID-1912132020(7...,COVID-19,COVID-1912132020(76).jpg,COVID-19COVID-1912132020(76).jpg


In [21]:
df['source'] = 'fusion'
df['pred'] = 'unknown'
df['time'] = 'unknown'

In [22]:
df.to_csv('results/test_ehl_edited_base.csv', sep=',', header=True, index=False, columns=['id', 'path', 'condition', 'source', 'pred', 'time'])

In [23]:
df.shape

(441, 7)

In [ ]:
# id,path,condition,source,split,pred,time